In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

2024-02-20 09:49:07.760212: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-20 09:49:07.783888: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-20 09:49:07.783909: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-20 09:49:07.783933: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-20 09:49:07.788942: I tensorflow/tsl/cuda/cudart_stub.cc:28] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-20 09:49:07.789351: I tensorflow/core/platform/cpu_feature_guard.cc:182] This Tens

# Preprocessing

In [31]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import os
import glob

# input and output directories
root_dir   = '/home/joshuavargas/research/datasets/BB-MAS_Dataset/BB-MAS_Dataset/'
output_dir = '/home/joshuavargas/research/datasets/BB-MAS_Preprocessed/'

# folders 1 - 117
for i in range(1, 118):
    folder_path = os.path.join(root_dir, str(i))
    
    # Assuming there's only one CSV file per folder, use glob to find CSV files
    csv_files = glob.glob(os.path.join(folder_path, '*_PocketPhone_*.csv'))
    
    if not csv_files:
        print(f"No PocketPhone files in directory {i}")
        continue
    
    for csv_file in csv_files:
        #print(csv_file)
        df = pd.read_csv(csv_file)
        print("Before scaling:")
        print(df.describe())
        output_df = df[['EID', 'time']].copy()
        
        scaler = MinMaxScaler(feature_range = (-1, 1))
        
        for column in ['Xvalue', 'Yvalue', 'Zvalue']:
            scaled_values = scaler.fit_transform(df[[column]])
            output_df[column] = scaled_values
            
        print("After scaling:")
        print(output_df.describe())
        
  
    #print(f"Processed folder {i}")


Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  34318.000000  34318.000000  34318.000000  34318.000000
mean   17158.500000      0.017913     -0.051102      0.032467
std     9906.897606      0.834573      0.750844      1.056525
min        0.000000     -3.944674     -4.907673     -3.967045
25%     8579.250000     -0.432497     -0.569916     -0.755273
50%    17158.500000      0.064981     -0.036219     -0.173105
75%    25737.750000      0.585895      0.451672      0.714792
max    34317.000000      2.984871      4.387824      4.059723
After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  34318.000000  34318.000000  34318.000000  34318.000000
mean   17158.500000      0.143679      0.044930     -0.003456
std     9906.897606      0.240874      0.161550      0.263250
min        0.000000     -1.000000     -1.000000     -1.000000
25%     8579.250000      0.013682     -0.066697     -0.199735
50%    17158.500000      0.157264      

Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  33876.000000  33876.000000  33876.000000  33876.000000
mean   16937.500000     -0.005630      0.021925      0.032878
std     9779.303196      0.899954      0.778088      0.812370
min        0.000000     -2.176335     -3.981959     -2.548113
25%     8468.750000     -0.655138     -0.412257     -0.559264
50%    16937.500000     -0.170442      0.073503     -0.121440
75%    25406.250000      0.498544      0.499609      0.547546
max    33875.000000      3.481284      3.763579      3.157444
After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  33876.000000  33876.000000  33876.000000  33876.000000
mean   16937.500000     -0.232644      0.033856     -0.095271
std     9779.303196      0.318139      0.200913      0.284764
min        0.000000     -1.000000     -1.000000     -1.000000
25%     8468.750000     -0.462248     -0.078256     -0.302838
50%    16937.500000     -0.290906      

Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  31500.000000  31500.000000  31500.000000  31500.000000
mean   15749.500000     -1.447045      9.673332     -0.611797
std     9093.411076      2.589094      4.497839      3.466455
min        0.000000    -27.223270     -6.809110    -31.082724
25%     7874.750000     -2.938883      7.485472     -2.242170
50%    15749.500000     -1.262941      9.607931     -0.325611
75%    23624.250000      0.203507     11.935694      0.948104
max    31499.000000     10.712855     37.039497     22.498314
After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  31500.000000  31500.000000  31500.000000  31500.000000
mean   15749.500000      0.358928     -0.248211      0.137377
std     9093.411076      0.136498      0.205153      0.129391
min        0.000000     -1.000000     -1.000000     -1.000000
25%     7874.750000      0.280278     -0.348003      0.076521
50%    15749.500000      0.368634     -

Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  29590.000000  29590.000000  29590.000000  29590.000000
mean   14794.500000      1.262398    -10.290263     -0.625692
std     8542.041569      3.460931      5.152325      6.806256
min        0.000000    -20.379444    -39.226601    -39.226601
25%     7397.250000     -0.505177    -12.886790     -2.923320
50%    14794.500000      1.037886     -9.848549      0.012570
75%    22191.750000      2.676718     -7.228095      2.788946
max    29589.000000     39.225403      8.899248     39.225403
After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  29590.000000  29590.000000  29590.000000  29590.000000
mean   14794.500000     -0.273823      0.202528     -0.015936
std     8542.041569      0.116129      0.214119      0.173514
min        0.000000     -1.000000     -1.000000     -1.000000
25%     7397.250000     -0.333133      0.094622     -0.074510
50%    14794.500000     -0.281356      

Before scaling:
               EID        Xvalue        Yvalue        Zvalue
count  33721.00000  33721.000000  33721.000000  33721.000000
mean   16860.00000      0.579839     -1.966859     -0.465474
std     9734.55855      4.515757     10.395836      3.289275
min        0.00000    -26.606764    -39.226601    -31.944635
25%     8430.00000     -1.735796     -9.928755     -1.786075
50%    16860.00000      0.016759     -6.232107     -0.019154
75%    25290.00000      2.317587      8.165425      1.199495
max    33720.00000     39.225403     39.225403     38.098930
After scaling:
               EID        Xvalue        Yvalue        Zvalue
count  33721.00000  33721.000000  33721.000000  33721.000000
mean   16860.00000     -0.174063     -0.050126     -0.101155
std     9734.55855      0.137190      0.265024      0.093921
min        0.00000     -1.000000     -1.000000     -1.000000
25%     8430.00000     -0.244413     -0.253101     -0.138863
50%    16860.00000     -0.191170     -0.158862     -0.

...

Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  28439.000000  28439.000000  28439.000000  28439.000000
mean   14219.000000      0.072397     -0.000708     -0.006807
std     8209.776489      1.772328      1.676841      0.617284
min        0.000000     -5.323126     -6.499178     -2.909237
25%     7109.500000     -1.247425     -1.127050     -0.388822
50%    14219.000000     -0.255663     -0.013848     -0.003196
75%    21328.500000      1.163269      1.108940      0.368582
max    28438.000000      5.869607      5.960155      2.593919
After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  28439.000000  28439.000000  28439.000000  28439.000000
mean   14219.000000     -0.035888      0.043149      0.054824
std     8209.776489      0.316693      0.269170      0.224338
min        0.000000     -1.000000     -1.000000     -1.000000
25%     7109.500000     -0.271724     -0.137654     -0.084011
50%    14219.000000     -0.094508      

Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  29211.000000  29211.000000  29211.000000  29211.000000
mean   14605.000000     -0.002339      0.041734      0.006551
std     8432.633693      1.158383      2.022297      1.584634
min        0.000000     -6.010222    -12.217518     -6.525810
25%     7302.500000     -0.702009     -0.905475     -0.749414
50%    14605.000000      0.030893      0.098004      0.060720
75%    21907.500000      0.626375      1.134507      0.707336
max    29210.000000      5.985721     11.987421      7.078682
After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  29211.000000  29211.000000  29211.000000  29211.000000
mean   14605.000000      0.001653      0.012955     -0.039676
std     8432.633693      0.193129      0.167098      0.232958
min        0.000000     -1.000000     -1.000000     -1.000000
25%     7302.500000     -0.114999     -0.065311     -0.150810
50%    14605.000000      0.007193      

After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  33837.000000  33837.000000  33837.000000  33837.000000
mean   16918.000000      0.081862      0.078081     -0.059272
std     9768.044866      0.127744      0.152126      0.071824
min        0.000000     -1.000000     -1.000000     -1.000000
25%     8459.000000      0.003841     -0.017545     -0.086900
50%    16918.000000      0.075895      0.067964     -0.051286
75%    25377.000000      0.153377      0.149716     -0.021469
max    33836.000000      1.000000      1.000000      1.000000
Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  18149.000000  18149.000000  18149.000000  18149.000000
mean    9074.000000      1.260441    -10.000151     -0.944614
std     5239.309353      2.732973      5.576015      4.991490
min        0.000000    -16.658855    -39.226601    -39.226601
25%     4537.000000     -0.265756    -12.710816     -2.458845
50%     9074.000000      1.231817     -

                EID        Xvalue        Yvalue        Zvalue
count  28978.000000  28978.000000  28978.000000  28978.000000
mean   14488.500000     -0.241636     -0.400986     -0.343979
std     8365.372387      0.196317      0.218318      0.101208
min        0.000000     -1.000000     -1.000000     -1.000000
25%     7244.250000     -0.373132     -0.554572     -0.406453
50%    14488.500000     -0.272336     -0.413558     -0.348035
75%    21732.750000     -0.141589     -0.290840     -0.293172
max    28977.000000      1.000000      1.000000      1.000000
Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  28978.000000  28978.000000  28978.000000  28978.000000
mean   14488.500000     -0.000869     -0.024984      0.017563
std     8365.372387      0.727381      0.966008      0.358250
min        0.000000     -3.022155     -4.233361     -2.752643
25%     7244.250000     -0.463124     -0.727576     -0.213053
50%    14488.500000     -0.101200      0.009587      0

Before scaling:
                EID        Xvalue        Yvalue        Zvalue
count  29397.000000  29397.000000  29397.000000  29397.000000
mean   14698.000000      1.513023      0.577114      0.294441
std     8486.327268      2.357237      9.702235      3.200315
min        0.000000     -9.743204    -26.120741    -16.803705
25%     7349.000000     -0.080206     -8.990228     -1.866280
50%    14698.000000      1.216254      4.841076      0.497994
75%    22047.000000      2.706645      9.221268      2.418144
max    29396.000000     14.069527     38.222233     20.646399
After scaling:
                EID        Xvalue        Yvalue        Zvalue
count  29397.000000  29397.000000  29397.000000  29397.000000
mean   14698.000000     -0.054604     -0.170139     -0.086884
std     8486.327268      0.197981      0.301579      0.170911
min        0.000000     -1.000000     -1.000000     -1.000000
25%     7349.000000     -0.188417     -0.467525     -0.202276
50%    14698.000000     -0.079529     -